# Machine Learning 2023-2024 - UMONS

# Linear Regression

**In this lab about linear regression, we'll be working with the library [statsmodels](https://www.statsmodels.org/stable/index.html), which provides numerous classes and functions for the estimation of statistical models.**

**The dataset that we'll be considering is [Diamonds](https://www.kaggle.com/datasets/shivam2503/diamonds), which contains several characteristics of a series of diamonds, such as their dimensions, the quality of their cuts, their prices, etc... The goal of the lab will be to define linear regression models to best estimate diamonds prices using a bunch of predictor variables, and to understand the meaning of the obtained coefficients.**

**Dataset's column information:**

- 'price': price in US dollars.
- 'carat': weight of the diamond. 
- 'cut': quality of the cut (Fair, Good, Very Good, Premium, Ideal)
- 'color': diamond's color's, from J (worst) to D (best).
- 'clarity': how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
- 'x': length in mm.
- 'y': width in mm. 
- 'z': height in mm.
- 'table': width of top of the diamond relative to its widest point. 
- 'depth' = 2z/(x+y) 

**Import the necessary libraries**

In [1]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from patsy import dmatrices
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn import datasets

**We load the 'Diamond' dataset as a Dataframe. We also add two columns with noise in order to experiment with variable selection.**

In [2]:
df = pd.read_csv('data/diamonds.csv', index_col=0)
# Add a column with noise
df['noise'] = np.random.normal(0, 1, len(df))
df['x_noise'] = df['x'] + np.random.normal(0, 1, len(df))
df

**1) Check the number of observations, the number of columns, the column types, and the number of missing values per columns.**

## Data cleaning

**2) Set the data type of the categorical variables to 'category'.**

**3) Make a scatter plot of the variable 'price' along the y-axis and the variable 'x' along the x-axis. Do the same for the variables 'y' and 'z'.**

**Do you notice anything special?**

**4) Following your observations from the previous question, what would you do?**

## Linear regression using statsmodels

**5) Using the [`dmatrices`](https://patsy.readthedocs.io/en/latest/API-reference.html#patsy.dmatrices) function of the patsy library and the [`OLS`](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html) class of the statsmodels library, fit a linear regression model to the diamonds' price using the variable 'x' as your predictor. Then, based on the results given by the [`summary`](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLSResults.summary.html) method:** 

- **5.1) Report the coefficient of determination and the adjusted coefficient of determination of the model.**
- **5.2) How would you write the linear regression model?**
- **5.3) Are the fitted coefficients statistically significant at the 5% significance level?**
- **5.4) How would you obtain the confidence intervals?**
- **5.5) How do you interpret the model?**

**6) Compute the linear regression coefficients and intercept using the formula $\hat{\beta} = (X^T X)^{-1} X^T y$. Check that the obtained coeffients are the same than in the previous exercise.**

**7) Fit the diamonds' price to the variable 'cut'. Note that 'cut' is a categorical variable, which we learned to preprocess in previous labs. When calling `dmatrices`, categorical variables will automatically be converted into dummy variables. You can also use the syntax [`C(variable)`](https://patsy.readthedocs.io/en/latest/categorical-coding.html) for more flexibility.**
- **7.1) Does the model appear to be a better fit than with the variable 'x'?**
- **7.2) How would you write the linear regression model now?**
- **7.3) Are all coefficients statistically significant at the 5% significance level?**
- **7.4) How do you interpret the model ? Does the intercept have a meaning?**
- **7.5) Using `sns.boxenplot`, compare the distributions of the variable 'price' for the different values of 'cut'. Is it coherent to the model results?**  

**8) Fit a linear regression model to the variable 'price' using 'x' and 'cut' as predictors. Based on [patsy's formulas](https://patsy.readthedocs.io/en/latest/formulas.html), add an interaction term between 'x' and 'cut'.**
- **8.1) How does the model write now?**
- **8.2) How do you interpret the model for a 'Good' cut diamond?**
- **8.3) Are all coefficients statistically significant ? If no, what does it mean?**

**9) Fit a linear regression model to the variable 'price' using all predictors.**
- **9.1) Derive the value of the Bayesian Information Criterion (BIC) and check that it corresponds to the value reported by the model's summary**  
- **9.2) Does the model appear to be a better fit than the ones using only 'x' and 'cut' as predictors? Which criterion would you look at?**
  

**10) The function `forward_selection` below implements a forward stepwise selection of predictors for linear regression. Make sure to understand each of its steps, and how they relate to the algorithm seen in class.**

**Using this function, find the best subset of predictors using the BIC as selection critierion. Redo the same by using the AIC and $R^2$ as selection criterion. Do you notice any differences ?**

In [19]:
def forward_selection(df, target, criterion='BIC'):
    assert criterion in ['BIC', 'AIC', 'R2'], 'Unknown criterion !'
    # Iteratively, select the best feature to add to the model
    candidates = set(df.columns) - {target}
    ordered_predictors = []
    while len(candidates) > 0:
        rsquared_dict = {}
        for predictor in candidates:
            res = fit(df, ordered_predictors + [predictor], target)
            rsquared_dict[predictor] = res.rsquared
        best_predictor = max(rsquared_dict, key=rsquared_dict.get)
        ordered_predictors = ordered_predictors + [best_predictor]
        candidates.remove(best_predictor)
    print(f'Best to worst predictors: {ordered_predictors}')
    # Compute the scores obtained by adding each feature
    current_features = []
    scores = []
    for feature in ordered_predictors:
        current_features += [feature]
        res = fit(df, current_features, target)
        if criterion == 'BIC':
            scores.append(res.bic)
        elif criterion == 'AIC':
            scores.append(res.aic)
        elif criterion == 'R2':
            scores.append(res.rsquared)
    # Select the features that optimize the criterion
    if criterion in ['BIC', 'AIC']:
        num_final_features_to_keep = np.argmin(scores)
    else:
        num_final_features_to_keep = np.argmax(scores)
    final_features_to_keep = ordered_predictors[:num_final_features_to_keep + 1]
    best_final_score = scores[num_final_features_to_keep]
    return final_features_to_keep, best_final_score

def fit(df, features_to_try, target):
    formula = get_formula(features_to_try, target)
    y, X = dmatrices(formula, data=df, return_type='dataframe')
    mod = sm.OLS(y, X)
    res = mod.fit()
    return res

def get_formula(features_to_try, target):
    return target + ' ~ ' + ' + '.join(features_to_try)